# Pràctica - El marc de l'aprenentatge automàtic:

El procés d'aplicar tècniques d'aprenentatge és un procés que consta de cinc parts:

- Tractament de les dades: preparació del conjunt de dades, selecció de característiques, **obtenció dels conjunts d'entrenament / test**.

- **Selecció de la / les mètriques adients**.

- **Selecció de la tècnica d'aprenentatge automàtic**.

- **Avaluació del model**.

- **Ajustament del model**.


## Separació del conjunt de dades: validació creuada per avaluar el rendiment del nostre model

### Mètode _holdout_

Aquest mètode consisteix a separar el conjunt de dades en tres subconjunts diferents: entrenament, validació i _test_. El conjunt d'entrenament s'usa com és habitual és a dir per entrenar els diferents models. El conjunt de validació s'usa per seleccionar el millor dels models. El conjunt de _test_, que no usem en cap cas durant el procés d'entrenament, ens servirà per obtenir una idea poc esbiaixada de la capacitat del model d'adaptar-se a noves mostres, sobre aquest conjunt de dades serà sobre el qual obtindrem les mètriques finals del model.

El procés d'aplicació d'aquesta tècnica es pot veure en el següent gràfic:

![](imatges/holdout.png)

Aquest mètode encara que senzill d'emprar té un desavantatge, ja que el rendiment del model depen de com hem fet la partició de les dades.

### Mètode _K-Fold_

Aquesta tècnica és més robusta, ja que repetim el mètode anterior _k_ vegades en _k_ subconjunts del conjunt d'entrenament, per tant, obtenim _k_ models i el mateix nombre de mesures de rendiment. El resultat final s'obtè amb la mitjana de cada una de les repeticions realitzades, d'aquesta manera els resultats depenen manco de les particions que realitzem.

Aquesta tècnica normalment s'usa per obtenir els millors paràmetres del model a aplicar, es a dir trobar aquells paràmetres que maximitzen el rendiment de la mètrica que volem usar. Un cop que tenim els millors paràmetres, reentrenam el model emprant el conjunt d'entrenament complet i obtenim les mètriques amb el conjunt de _test_.

El procés d'aplicació d'aquesta tècnica es pot veure en el següent gràfic:

<img src="imatges/Kfold.png" alt="kfold" width="600"/>

La pregunta que ens podem fer és: Com seleccionar aquest paràmetre _k_ de forma correcta?

Finalment, existeix una variant d'aquesta tècnica anomenada _stratified k-fold_ en el que les proporcions entre classes es mantenen a cada una de les iteracions, això és important quan tenim problemes desbalancejats.

## Selecció de mètriques

Un cop entrenat el nostre model, tenim la necessitat d'avaluar els resultats obtinguts amb aquest amb alguna mesura que sigui objectiva. Les mesures que explicarem en aquesta secció es calculen a partir d'una matriu de confusió que ens permet guardar quatre mesures bàsiques a partir de considerar que una de les classes és la positiva i l'altra és la negativa.

- _True Positives_ (TP): L'algorisme classifica una mostra de la classe positiva com a membre de la classe positiva.
- _True Negatives_ (TN): L'algorisme classifica una mostra de la classe negativa com a membre de la classe negativa.
- _False Positives_ (FP): L'algorisme classifica una mostra de la classe negativa com a membre de la classe positiva.
- _False Negatives_ (FN): L'algorisme classifica una mostra de la classe positiva com a membre de la classe negativa.

Podem observar la matriu de confusió en el següent esquema:

![image](imatges/confusion_matrix.png "font: Python Machine Learning")

Aquesta matriu es pot obtenir de manera senzilla usant la funció `confusion_matrix` de la llibreria [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html?highlight=confusion%20matrix#sklearn-metrics-confusion-matrix)
i es pot visualitzar amb la funció [ConfusionMatrixDisplay](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html?highlight=confusion%20matrix#sklearn-metrics-confusionmatrixdisplay)

A partir d'aquestes mesures de primer ordre en podem treure d'altres més completes com l'error o l'exactitud, també es coneix com _Accuracy_.

$$ Error = \frac{FP+FN}{FP+FN+TP+TN}$$
<br>
$$ Exactitut = \frac{TP+TN}{FP+FN+TP+TN} = 1 - Error$$

Per altra banda, tenim les mesures Rati de Vertaders Positius (_True Positive Rate_, TPR) i la Ratio de Falsos Positius (_False Positive Rate_, FPR) que estan dissenyades per problemes on hi ha una classe amb més mostres que l'altra:

$$ FPR = \frac{FP}{N} = \frac{FP}{FP+TN} $$
<br>
$$ TPR = \frac{TP}{P} = \frac{TP}{FN+TP} $$

Finalment, parlarem de precisió (_precision_) i la sensibilitat (_recall_) relacionades amb les ratios de vertaders positius i vertaders negatius:

$$ Precisio = \frac{TP}{TP+FP}$$
<br>
 $$ Sensibilitat = TPR = \frac{TP}{FN+TP} $$

Tenim una mesura que engloba aquestes mesures anteriors:

$$ F1 = 2 \frac{Precisio \times Sensibilitat}{Precisio + Sensibilitat}$$

Per sort tenim un mòdul anomenat _metrics_ on hi ha totes aquestes (i d'altres) mètriques ja implementades.

## Un exemple complet

A continuació teniu un exemple que resumeix el procés sencer emprant la llibreria `Scikit-learn`:

In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

X, y = make_classification(n_samples=1000, n_features=2, n_redundant=0, n_repeated=0,
                           n_classes=2, n_clusters_per_class=1, class_sep=2,
                           random_state=5)

# Tractament de les dades: Separació i estandaritzat
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenament i predicció, aquí no hi ha ajustament de paràmetres
clf = SGDClassifier(loss="perceptron", eta0=1, max_iter=1000, learning_rate="constant", random_state=5)
clf.fit(X_train_scaled, y_train)
prediction = clf.predict(X_test_scaled)

# Avaluacio
cf_matrix = confusion_matrix(y_test, prediction)
print(cf_matrix)
accuracy = accuracy_score(y_test, prediction)
print("Accuracy: ", accuracy)

[[114   3]
 [  1  82]]
Accuracy:  0.98



## Ajustar el model usant una cerca exhaustiva (_Grid Search_)

Els paràmetres del nostre model que podem ajustar manualment, es a dir que no són apresos de les dades d'entrenament, s'anomenen hiper-paràmetres. Ajustar el seu valor de manera intuïtiva o mitjançant successions de proves i errors pot ser una tasca llarga, per no dir impossible en el cas de models amb molts paràmetres com poden ser els _Random Forests_.

Existeix una tècnica de cerca exhaustiva dels valors òptims dels hiper-paràmetres coneguda amb el nom de _Grid Search_ que ens permet automatitzar aquesta cerca penalitzant el cost temporal del procés d'entrenament.


## Combinació de _K-Fold_ amb _Grid Search_

La combinació de les dues tècniques explicades anteriorment és una de les més emprades, és coneix amb el nom de
_nested cross-validation_. En aquest cas tenim dos bucles, un dins l'altra: el més extern en el que es divideix el conjunt
d'entrenament usant _K-folding_ i un altre d'intern en el qual es realitza la cerca dels millors hiper-paràmetres.

L'esquema que segueix és el següent:

<img src="imatges/grid_search_k_fold.png" width="600"/>

## Resum del procés complet

En aquesta pràctica aplicarem l'explicat fins ara usant les eines que _Scikit_ posa al nostre abast. Les passes a seguir són:

0. Creació del conjunt de dades.
1. Separació del conjunt de dades: entrenament i test.
2. Definició dels paràmetres a ajustar segons el model triat. El format de la graella de paràmetres és un diccionari on la clau és el nom del paràmetre i el valor una llista amb tots els valors a provar. [Diccionaris a Python](https://docs.python.org/3/tutorial/datastructures.html#dictionaries).
3. Aplicam la cerca exhaustiva (_grid search_) juntament amb _k-folding_. Usarem la funció _GridSearchCV_ [enllaç](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearchcv#sklearn.model_selection.GridSearchCV): `GridSearchCV(estimator, param_grid, cv=None, verbose=0)`.
4. Entrenam amb el millor model obtingut i el conjunt d'entrenament sencer. Podem obtenir el millor model resultant de la passa anterior amb l'atribut `best_estimator_` de l'objecte `GridSearchCV`. _Nota:_ Si mirau la documentació observareu que podem obtenir altres informacions del procés de la cerca exhaustiva.
5. Obtenir els resultats finals amb el conjunt de _test_.